<a href="https://colab.research.google.com/github/Syamabbas/All-About-Excel/blob/main/Analysis_Sentiment_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pandas Sastrawi openpyxl

In [19]:
import numpy
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [20]:
# ===============================
# 1. LOAD DATA
# ===============================
df = pd.read_excel("/content/drive/MyDrive/Google Collab/Analysis Sentiment/hasil_auto_labeling (1).xlsx")
TEXT_COL = "clean_text"   # sesuaikan jika nama kolom berbeda

In [21]:
# ===============================
# 2. Aspek
# ===============================
aspect_keywords = {
    "Waktu Pengiriman": [
        "cepat","cepet","lama","lambat","telat","terlambat","molor",
        "1 hari sampai","2 hari sampai","3 hari sampai","hari belum",
        "same day","next day","lelet","lemot","lemod","hambat",
        "estimasi","delay","pending","tertahan","stuck","tepat waktu",
        "belum sampai","tak kunjung sampai","nyampe","sampe",
        "tidak dikirim","tidak sesuai jaminan","transit lama","tidak gerak",
        "tolong ontime","tidak responsif","tidak prioritas","tidak jalan",
        "paket ngendap","ditahan","bolak balik","muter muter",
        "barang tertinggal","pengiriman cepat","extra cepat",
        "on time","tercepat","tidak diantar","belum dikirim"
    ],

    "Layanan SDM": [
        "kurir","driver","admin","cs","customer service","petugas",
        "pegawai","supir","armada","sdm","marketing","supirnya",
        "ramah","sopan","jutek","cuek","kasar","galak","ketus","mengirim",
        "arogan","attitude buruk","kurang etika","acuh","malas",
        "respon","slow respon","fast respon","pelayanan","kerja",
        "tidak dibalas","ga dibales","ga respon","tidak ada komunikasi",
        "sulit dihubungi","tidak profesional","telepon","di hubungin","hubungi",
        "mengeluh","kurir tidak paham","kurir ceroboh","mengantar",
        "kurir galak","admin galak","ugal-ugalan","nabrak","nyasar",
        "serempet","shaming","berisik","crewet","cerewet",
        "perlu training","perbaiki kurir","perbaiki admin","teliti",
        "pegawai sangat ramah","mantap sekali","sangat membantu",
        "petugas responsif","konfirmasi bagus","karyawan membantu",
        "kooperatif cepat","karyawan gesit","sabaran","dikirim ke orang lain"
    ],

    "Layanan Sistem": [
        "sistem","aplikasi","app","apps","website","web","situs",
        "bug","error","eror","crash","force close",
        "maintenance","down","email",
        "tracking","resi","status tidak update",
        "login gagal","logout sendiri"
    ],

    "Penanganan Barang": [
        "barang rusak","barang pecah","barang aman","barang hilang",
        "paket hilang","barang tertukar","barang terpisah",
        "isi kurang","salah kirim","aman sampai tujuan","paket rusak", "pecah",
        "packing","kemasan","packaging","box","paket saya hilang","paket hilang",
        "packing rusak","packing sobek","packing bagus",
        "volume barang","terselip","teledor","dibanting","lempar"
    ],

    "Komplain": [
        "buruk","jelek","parah","kecewa","kapok","nyesel","menyesal",
        "bikin emosi","review ancur","dulu bagus","perlu perbaikan",
        "pikir ulang","banyak alasan","tidak niat","cocok",
        "palsu","penipu","pencuri","ditipu","diblokir",
        "internal buruk","tidak profesional","korban","bintang",
        "toilet kotor","kotor","jorok","bosok","terima kasih",
        "tai","jancuk","asu","anjing","bangsat","memuaskan",
        "viralkan","ekspedisi engga jelas","ekspedisi parah","okay",
        "tidak terawat","aturan","mantap","tidak becus","anjing",
        "alhamdulillah","the best","terbaik","nyaman","payah","kacau",
        "aman","terpercaya","puas","bagus","mantap","mantul",
        "oke","tanggung jawab","recommended","rekomen","tempat besar",
        "rekomendasi","amanah","baik","tempat bersih","good","bersih"
    ],

    "Layanan Bisnis": [
        "promo","diskon","cashback","voucher",
        "harga","ongkos","ongkir","program","membership","member",
        "gratis ongkir","ongkir mahal","ongkir murah","ongkir kurang",
        "harga mahal","harga murah","harga tidak sesuai",
        "tambahan ongkir","harus bayar lagi","tidak tercover",
        "volume tidak sesuai","bintang satu",
        "worth it","tidak worth it","murah","terjangkau",
        "harga bersahabat","harga bersaing","sangat ekonomis",
        "ongkos tidak mahal","cargo murah",
        "pelayanan baik","pelayanan konsisten","service bagus",
        "ok juga","sentral cargo terbaik","mantap pelayanan",
        "mantap bosku","pertahankan ya","strategis",
        "langganan","lokasi aman","lokasi mudah",
        "andalan","dapat diandalkan"
    ]
}

In [22]:
# ===============================
# 3. PRIORITY ORDER
# ===============================
ASPECT_PRIORITY_ORDER = [
    "Waktu Pengiriman",
    "Penanganan Barang",
    "Layanan SDM",
    "Layanan Sistem",
    "Layanan Bisnis",
    "Komplain"
]

In [23]:
# ===============================
# 4. SLANG NORMALIZATION
# ===============================
SLANG_MAP = {
    "ga": "tidak", "gak": "tidak", "gk": "tidak", "ndak": "tidak", "kagak": "tidak", "tdk": "tidak",
    "udah": "sudah", "udh": "sudah", "sdh": "sudah", "dulu": "dahulu",
    "cepet": "cepat", "cpt": "cepat", "cepetan": "cepat",
    "lama": "lambat", "lemot": "lambat", "lelet": "lambat",
    "brg": "barang", "barangnya": "barang",
    "jg": "juga", "jga": "juga",
    "klo": "kalau", "kalo": "kalau", "klw": "kalau",
    "bgt": "banget", "buanget": "banget",
    "sampe": "sampai", "nyampe": "sampai", "smpe": "sampai",
    "pk": "pakai", "pake": "pakai", "pakek": "pakai",
    "tlp": "telepon", "tlvn": "telepon", "telpon": "telepon", "no": "nomor", "nomer": "nomor",
    "bales": "balas", "dibales": "dibalas", "respon": "respons",
    "mulu": "terus", "trus": "terus",
    "admin": "admin", "min": "admin",
    "kurir": "kurir", "kurirnya": "kurir",
    "banget": "sekali", "bngt": "sekali",
    "mantap": "bagus", "mantul": "bagus", "oke": "bagus", "ok": "bagus",
    "kecewa": "buruk", "parah": "buruk", "jelek": "buruk",
    "ongkir": "ongkos kirim", "cod": "cash on delivery",
    "hub": "hubung", "hubungi": "hubung",
}

def normalize_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)

    words = text.split()
    words = [SLANG_MAP.get(w, w) for w in words]

    return " ".join(words)


In [24]:
# ===============================
# 5. STEMMING
# ===============================
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
    return stemmer.stem(text)


In [25]:
# ===============================
# 6. PREPROCESS KEYWORDS
# ===============================
def preprocess_keywords(aspect_keywords):
    processed = {}
    for aspect, keywords in aspect_keywords.items():
        processed[aspect] = []
        for kw in keywords:
            kw_norm = normalize_text(kw)
            kw_stem = stem_text(kw_norm)
            processed[aspect].append(kw_stem)
    return processed

optimized_aspect_keywords = preprocess_keywords(aspect_keywords)


In [26]:
# ===============================
# 7. ASPECT CLASSIFIER + PRIORITY
# ===============================
def classify_aspect_with_priority(text, aspect_dict, priority_order):
    if pd.isna(text):
        return "Tidak Terklasifikasi", "Tidak Terklasifikasi"

    text_norm = normalize_text(text)
    text_stem = stem_text(text_norm)

    aspect_scores = {}

    for aspect, keywords in aspect_dict.items():
        score = 0
        for kw in keywords:
            if re.search(r"\b" + re.escape(kw) + r"\b", text_stem):
                score += 1
        if score > 0:
            aspect_scores[aspect] = score

    if not aspect_scores:
        return "Tidak Terklasifikasi", "Tidak Terklasifikasi"

    # multi-aspect
    all_aspects = "; ".join(sorted(aspect_scores.keys()))

    # priority logic
    max_score = max(aspect_scores.values())
    top_aspects = [a for a, s in aspect_scores.items() if s == max_score]

    priority_aspect = "Tidak Terklasifikasi"
    for aspect in priority_order:
        if aspect in top_aspects:
            priority_aspect = aspect
            break

    return all_aspects, priority_aspect


In [27]:
# ===============================
# 8. APPLY KE DATAFRAME
# ===============================
df[["Aspek", "Aspek_Priority"]] = df[TEXT_COL].apply(
    lambda x: pd.Series(
        classify_aspect_with_priority(
            x,
            optimized_aspect_keywords,
            ASPECT_PRIORITY_ORDER
        )
    )
)

In [28]:
# ===============================
# 9. SAVE OUTPUT
# ===============================
df.to_excel("/content/drive/MyDrive/Google Collab/Analysis Sentiment/Hasil Analysis GPT.xlsx", index=False)

print("✅ Aspect & Aspect Priority berhasil ditambahkan")

✅ Aspect & Aspect Priority berhasil ditambahkan
